"""
core2_02_synthetic_mutation_generator.ipynb

목적:
- 실제 생물학적으로 '그럴듯한' mutation을 만드는 것이 목적이 아니다.
- Core 2에서는 '설계 개입이 발생했다'는 사실과
  그 개입이 누적되는 구조를 관찰하기 위한 형식적 mutation을 생성한다.

원칙:
- mutation은 서열 변경 이벤트로만 취급한다.
- 생물학적 타당성, 에너지 안정성, 구조 고려는 Core 2에서 배제한다.
"""

In [1]:
import random
import pandas as pd
from pathlib import Path

random.seed(42)

# Core 1에서 고정된 항체 후보
ANTIBODY_CANDIDATES = {
    "A_boundary": {
        "antibody_key": "GDPa1-001",
        "name": "abagovomab",
        "sequence": None  # 실제 서열은 CSV 또는 별도 파일에서 로딩 가능
    },
    "B_conflict": {
        "antibody_key": "GDPa1-045",
        "name": "cixutumumab",
        "sequence": None
    },
    "C_obvious_risk": {
        "antibody_key": "GDPa1-183",
        "name": "prolgolimab",
        "sequence": None
    }
}

print("Core 2 mutation 대상 항체:")
for k, v in ANTIBODY_CANDIDATES.items():
    print(f"- {k}: {v['antibody_key']} / {v['name']}")

Core 2 mutation 대상 항체:
- A_boundary: GDPa1-001 / abagovomab
- B_conflict: GDPa1-045 / cixutumumab
- C_obvious_risk: GDPa1-183 / prolgolimab


In [2]:
# 형식적 기준 서열 생성 (길이만 중요->개입로그용)
def generate_dummy_sequence(length=120):
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")
    return "".join(random.choice(amino_acids) for _ in range(length))

for v in ANTIBODY_CANDIDATES.values():
    v["sequence"] = generate_dummy_sequence()

In [3]:
def generate_single_mutation(sequence, step, mutation_id_prefix):
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")
    
    pos = random.randint(0, len(sequence) - 1)
    from_aa = sequence[pos]
    
    to_aa = random.choice([aa for aa in amino_acids if aa != from_aa])
    
    sequence_after = (
        sequence[:pos] + to_aa + sequence[pos+1:]
    )
    
    return {
        "step": step,
        "mutation_id": f"{mutation_id_prefix}_{step}",
        "mutation_type": "substitution",
        "mutation_pos": pos,
        "from_aa": from_aa,
        "to_aa": to_aa,
        "sequence_before": sequence,
        "sequence_after": sequence_after
    } # Synthetic Mutation 후보 생성 함수

	•	“발생한 mutation”이 아니라
	•	“발생 가능했던 mutation 후보” 세트

In [4]:
MUTATION_CANDIDATES = [] # 항체별 mutation 후보 세트 생성

MAX_STEPS = 20  # Core 2에서 볼 설계 단계 수
CANDIDATES_PER_STEP = 3  # 각 step마다 가능한 mutation 수

for group_key, info in ANTIBODY_CANDIDATES.items():
    base_seq = info["sequence"]
    antibody_key = info["antibody_key"]
    
    current_seq = base_seq
    
    for step in range(1, MAX_STEPS + 1):
        for i in range(CANDIDATES_PER_STEP):
            mut = generate_single_mutation(
                sequence=current_seq,
                step=step,
                mutation_id_prefix=f"{antibody_key}_mut{i}"
            )
            
            mut["antibody_key"] = antibody_key
            mut["candidate_group"] = group_key
            
            MUTATION_CANDIDATES.append(mut)

In [5]:
mutation_df = pd.DataFrame(MUTATION_CANDIDATES)

mutation_df.head() # mutation_log DataFrame 생성

,step,mutation_id,mutation_type,mutation_pos,from_aa,to_aa,sequence_before,sequence_after,antibody_key,candidate_group
0,1,GDPa1-001_mut0_1,substitution,62,P,K,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001,A_boundary
1,1,GDPa1-001_mut1_1,substitution,115,E,C,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001,A_boundary
2,1,GDPa1-001_mut2_1,substitution,11,C,R,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,EAKIIFEVDWQRADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001,A_boundary
3,2,GDPa1-001_mut0_2,substitution,106,W,K,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001,A_boundary
4,2,GDPa1-001_mut1_2,substitution,5,F,A,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,EAKIIAEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001,A_boundary


In [6]:
print("mutation_log.csv 컬럼 구조:\n")
for col in mutation_df.columns:
    print("-", col)

print("\n총 mutation 후보 수:", len(mutation_df)) # 컬럼 구조 확인 

mutation_log.csv 컬럼 구조:

- step
- mutation_id
- mutation_type
- mutation_pos
- from_aa
- to_aa
- sequence_before
- sequence_after
- antibody_key
- candidate_group

총 mutation 후보 수: 180


In [7]:
output_dir = Path("../artifact/core2")
output_dir.mkdir(parents=True, exist_ok=True)

output_path = output_dir / "mutation_candidate_set.csv"
mutation_df.to_csv(output_path, index=False)

print(f"Mutation candidate set saved to:\n{output_path.resolve()}") # 파일 저장

Mutation candidate set saved to:
/Users/mac/Desktop/De/Developability_Data/core/artifact/core2/mutation_candidate_set.csv


"""
[Core 2 Mutation Generation Declaration]

- 본 mutation set은 생물학적 타당성을 목표로 하지 않는다.
- 이 mutation들은 '설계 개입이 가능했던 선택지 공간'을 나타낸다.
- 실제 mutation 발생 여부는 Core 2-03의 정책(policy)에 의해 결정된다.

즉,
- mutation 생성 ≠ mutation 실행
- 실행은 전적으로 '예측 중심 설계 정책'의 결과이다.
"""